In [1]:
from ToolsMoodleXML2 import *
import codecs, glob

In [2]:
def makeTitre(titre):
    titre=[
            u'<div style="text-align: center;"><span style="font-size: x-large;">',
            titre,
            u'</span><br></div>'
        ]
    return "\n".join(titre)


# Assemblage des extraits txt

In [3]:
texteExtraits={}
listeExtraitNoms=glob.glob("/Users/gilles/Dropbox/Rugby/TestSyntaxeDiscours/E*.txt")
for numNom,nomFichier in enumerate(listeExtraitNoms):
    with codecs.open(nomFichier,"r",encoding="utf8") as input:
        extraitLignes=input.readlines()
    texteExtraits[numNom+1]=r"<br/>".join([l.strip() for l in extraitLignes])

In [4]:
#texteExtraits[1]

## Découpage des frames

In [5]:
nomFichierSCF="/Users/gilles/Dropbox/Rugby/TestSyntaxeDiscours/SoundCloud-Frames.txt"
with codecs.open(nomFichierSCF,"r", encoding="utf8") as input:
    frames=input.readlines()

### associer les extraits et les numéros de track

In [6]:
trackExtrait={}
numExtrait=1
for frame in frames:
    if frame.startswith(r"<iframe"):
        numTrack=frame.split("tracks/")[1].split("&")[0]
        trackExtrait[numExtrait]=numTrack
        numExtrait+=1

### déclarer le format des iframes pour SoundCloud

In [7]:
frameDeb=r'<iframe width="100%" height="166" scrolling="no" frameborder="no" src="https://w.soundcloud.com/player/?url=https%3A//api.soundcloud.com/tracks/'
frameFin=r'&amp;color=ff9900&amp;auto_play=false&amp;hide_related=true&amp;show_comments=false&amp;show_user=false&amp;show_reposts=false"></iframe>'

## Découpage des consignes

In [8]:
nomFichierConsignes="/Users/gilles/Dropbox/Rugby/TestSyntaxeDiscours/SCD-consignes.txt"
with codecs.open(nomFichierConsignes,"r",encoding="utf8") as input:
    lineConsignes=input.readlines()

In [9]:
consignes={}
subCat=None
for ligne in lineConsignes:
    if ligne.startswith("="):
        subCat=ligne.split("=")[1].lower()
        if subCat in consignes:
            print "la catégorie %s est déjà définie"%subCat
        else:
            consignes[subCat]=[]
    else:
        if subCat:
            consignes[subCat].append(ligne.strip())

## Assemblage du Moodle XML

In [10]:
def remplirEssay():
    for consigne in consignes:
        questions.addExercice(XMLSubCategory(categorie+"/%s"%consigne))
        for num in trackExtrait:
#            print num
            questionText=makeTitre(u"Texte à découper")
            questionText+="<hr>"
            questionText+=texteExtraits[num]+u"<br/><hr>\n"
#            print frameDeb+trackExtrait[num]+frameFin
            questionText+=makeTitre(u"Consigne")
            questionText+=u"<br/>\n".join(consignes[consigne])+"<br><hr>"
            questionText+=makeTitre(u"Enregistrement")
            questionText+=frameDeb+trackExtrait[num]+frameFin
            questions.addExercice(Essay("extrait-%02d %s"%(num,consigne),questionText))



In [11]:
categorie="TestDecoupage"
questions=XMLSeries(categorie)
remplirEssay()
xmlOutput=MoodleXML()
xmlOutput.addQuiz(questions.getSeries())
print xmlOutput.getXML()

<?xml version="1.0" encoding="UTF-8"?>
<quiz>
<question type="category">
<category>
<text>
TestDecoupage
</text>
</category>
</question><question type="category">
<category>
<text>
TestDecoupage/phrases
</text>
</category>
</question><question type="essay">
<name><text>extrait-01 phrases</text></name>
<questiontext><text><![CDATA[<div style="text-align: center;"><span style="font-size: x-large;">
Texte à découper
</span><br></div><hr>﻿après avoir demandé si tout le monde était prêt le coup d'envoi de la coupe du monde deux mille sept est donné par David Skrela allez de suite dans le camp euh argentin avec un départ de Roncero Pichot a appelé le ballon pour Hernandez avec le pied droit pour trouver une très jolie touche direct il en avait le droit il était à l'intérieur de ses vingt-deux mètres<br/>dans ce cas-là on peut trouver une touche directement alors la touche ça ça sera une des clefs de cette rencontre le bloc français contre le bloc argentin<br/><br/>lancer pour le capitaine de

In [12]:
xmlFichier=nomFichierConsignes.replace(".txt",".xml")
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
    output.write(xmlOutput.getXML())